In [ ]:
def Process(adata_ref,path,output_path,prefix,suffix,filename):
    import sys
    IN_COLAB = "google.colab" in sys.modules
    if IN_COLAB and branch == "stable":
        !pip install --quiet git+https://github.com/BayraktarLab/cell2location#egg=cell2location[tutorials]
    import scanpy as sc
    import anndata
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    import cell2location
    import scvi
    from matplotlib import rcParams
    rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
    import seaborn as sns
    import loompy
    import pyreadr
    import os
    ## load simulated ST data counts,transfer it to anndata and preprocess.
    result = pyreadr.read_r(f'{path}{prefix}/{filename}')
    df = result[None]
    df2 = df.T
    n_obs = len(df2.index.values)
    obs = pd.DataFrame()
    obs['orig.ident'] = np.random.choice(['simulated_data'], n_obs)
    obs['sample'] = np.random.choice([prefix], n_obs)
    obs.index = df2.index.values
    var_names = df2.columns.values
    n_vars = len(var_names)
    var = pd.DataFrame(index=var_names)
    adata_vis = anndata.AnnData(df2, obs=obs, var=var, dtype='int32')
    
    adata_vis.var['SYMBOL'] = adata_vis.var_names
    adata_vis.var['gene_ids'] = adata_vis.var_names
    adata_vis.var_names.name = None
    
    # find mitochondria-encoded (MT) genes
    adata_vis.var['MT_gene'] = [gene.startswith('MT-') for gene in adata_vis.var['SYMBOL']]
    
    # remove MT genes for spatial mapping (keeping their counts in the object)
    adata_vis.obsm['MT'] = adata_vis[:, adata_vis.var['MT_gene'].values].X.toarray()
    adata_vis = adata_vis[:, ~adata_vis.var['MT_gene'].values]
    
    ## Process the adata.vis
    if 'means_per_cluster_mu_fg' in adata_ref.varm.keys():
        inf_aver = adata_ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'                                                              
                                                              for i in adata_ref.uns['mod']['factor_names']]].copy()
    else:
        inf_aver = adata_ref.var[[f'means_per_cluster_mu_fg_{i}'
                                  for i in adata_ref.uns['mod']['factor_names']]].copy()
    inf_aver.columns = adata_ref.uns['mod']['factor_names']
    inf_aver.iloc[0:5, 0:5]
    # find shared genes and subset both anndata and reference signatures
    intersect = np.intersect1d(adata_vis.var_names, inf_aver.index)
    adata_vis = adata_vis[:, intersect].copy()
    inf_aver = inf_aver.loc[intersect, :].copy()

    # prepare anndata for cell2location model
    cell2location.models.Cell2location.setup_anndata(adata=adata_vis, batch_key="sample")

    # create and train the model
    mod = cell2location.models.Cell2location(
        adata_vis, cell_state_df=inf_aver,
        # the expected average cell abundance: tissue-dependent
        # hyper-prior which can be estimated from paired histology:
        N_cells_per_location=30,
        # hyperparameter controlling normalisation of
        # within-experiment variation in RNA detection (using default here):
        detection_alpha=200
    )
    
    mod.train(max_epochs=30000,
              # train using full data (batch_size=None)
              batch_size=None,
              # use all data points in training because
              # we need to estimate cell abundance at all locations
              train_size=1,      
              use_gpu=True)
    
    # plot ELBO loss history during training, removing first 100 epochs from the plot
    mod.plot_history(1000)
    plt.legend(labels=['full data training']);
    # In this section, we export the estimated cell abundance (summary of the posterior distribution).
    adata_vis = mod.export_posterior(
        adata_vis, sample_kwargs={'num_samples': 1000, 'batch_size': mod.adata.n_obs, 'use_gpu': True}
    )
    
    # Save model
    mod.save(f"{output_path}{prefix}_model_st_{suffix}.pt", overwrite=True)
    
    # mod = cell2location.models.Cell2location.load(f"{run_name}", adata_vis)
    
    # Save anndata object with results
    adata_file = f"{output_path}{prefix}_simu_{suffix}.h5ad"
    adata_vis.write(adata_file)
    return(adata_vis)



In [ ]:
def Preprocess(adata_ref,output_path,namesuffix):
    import sys
    IN_COLAB = "google.colab" in sys.modules
    if IN_COLAB and branch == "stable":
        !pip install --quiet git+https://github.com/BayraktarLab/cell2location#egg=cell2location[tutorials]
    import scanpy as sc
    import anndata
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib as mpl
    import cell2location
    import scvi
    from matplotlib import rcParams
    rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
    import seaborn as sns
    import loompy
    import pyreadr
    import os
    # Use ENSEMBL as gene IDs to make sure IDs are unique and correctly matched
    adata_ref.var['SYMBOL']=adata_ref.var.index
    adata_ref.var.index = adata_ref.var['GeneID-2'].copy()
    adata_ref.var_names = adata_ref.var['GeneID-2'].copy()
    adata_ref.var.index.name = None
    # adata_ref.raw.var['SYMBOL'] = adata_ref.raw.var.index
    # adata_ref.raw.var.index = adata_ref.raw.var['GeneID-2'].copy()
    # adata_ref.raw.var.index.name = None
    # before we estimate the reference cell type signature we recommend to perform very permissive genes selection
    # in this 2D histogram orange rectangle lays over excluded genes.
    # In this case, the downloaded dataset was already filtered using this method,
    # hence no density under the orange rectangle
    from cell2location.utils.filtering import filter_genes
    selected = filter_genes(adata_ref, cell_count_cutoff=5, cell_percentage_cutoff2=0.03, nonz_mean_cutoff=1.12)
    # filter the object
    adata_ref = adata_ref[:, selected].copy()
    # prepare anndata for the regression model
    if(namesuffix == 'KRAS'):    
        cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                                                           # 10X reaction / sample / batch
                                                           batch_key='patient',               
                                                           # cell type, covariate used for constructing signatures
                                                           labels_key='subclass',
                                                           # multiplicative technical effects (platform, 3' vs 5', donor effect)
                                                           categorical_covariate_keys=['orig.ident']
                                                          )
        

    elif(namesuffix == 'HL'):
        cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                                                           # 10X reaction / sample / batch
                                                           batch_key='Sample',                                                       
                                                           # cell type, covariate used for constructing signatures
                                                           labels_key='subclass',
                                                           # multiplicative technical effects (platform, 3' vs 5', donor effect)
                                                           categorical_covariate_keys=['Method']
                                                          )
    elif(namesuffix == 'stxB'):        
        cell2location.models.RegressionModel.setup_anndata(adata=adata_ref,
                                                           # 10X reaction / sample / batch
                                                           batch_key='sample_type',                                                       
                                                           # cell type, covariate used for constructing signatures
                                                           labels_key='subclass',
                                                           # multiplicative technical effects (platform, 3' vs 5', donor effect)
                                                           categorical_covariate_keys=['orig.ident']
                                                          )
    # create the regression model
    from cell2location.models import RegressionModel
    mod = RegressionModel(adata_ref)
    # view anndata_setup as a sanity check
    mod.view_anndata_setup()
    # create and train the regression model
    from cell2location.models import RegressionModel
    mod = RegressionModel(adata_ref)
    # Use all data for training (validation not implemented yet, train_size=1)
    mod.train(max_epochs=1000, use_gpu=True)
    # plot ELBO loss history during training, removing first 20 epochs from the plot
    mod.plot_history(20)
    # In this section, we export the estimated cell abundance (summary of the posterior distribution).
    adata_ref = mod.export_posterior(
        adata_ref, sample_kwargs={'num_samples': 1000, 'batch_size': 2500, 'use_gpu': True}
    )
    # Save model
    mod.save(f"{output_path}{namesuffix}_model_sc.pt", overwrite=True)
    # Save anndata object with results
    adata_file = f"{output_path}sc_{namesuffix}.h5ad"
    adata_ref.write(adata_file)
    return(adata_ref)
